In [2]:
import pandas as pd
import os

paths = []
for root, _, files in os.walk("./data"):
    for file in files:
        path = os.path.join(root, file)
        paths.append(path)

In [3]:
header_row = [
    "GLOBALEVENTID",
    "SQLDATE",
    "MonthYear",
    "Year",
    "FractionDate",
    "Actor1Code",
    "Actor1Name",
    "Actor1CountryCode",
    "Actor1KnownGroupCode",
    "Actor1EthnicCode",
    "Actor1Religion1Code",
    "Actor1Religion2Code",
    "Actor1Type1Code",
    "Actor1Type2Code",
    "Actor1Type3Code",
    "Actor2Code",
    "Actor2Name",
    "Actor2CountryCode",
    "Actor2KnownGroupCode",
    "Actor2EthnicCode",
    "Actor2Religion1Code",
    "Actor2Religion2Code",
    "Actor2Type1Code",
    "Actor2Type2Code",
    "Actor2Type3Code",
    "IsRootEvent",
    "EventCode",
    "EventBaseCode",
    "EventRootCode",
    "QuadClass",
    "GoldsteinScale",
    "NumMentions",
    "NumSources",
    "NumArticles",
    "AvgTone",
    "Actor1Geo_Type",
    "Actor1Geo_FullName",
    "Actor1Geo_CountryCode",
    "Actor1Geo_ADM1Code",
    "Actor1Geo_Lat",
    "Actor1Geo_Long",
    "Actor1Geo_FeatureID",
    "Actor2Geo_Type",
    "Actor2Geo_FullName",
    "Actor2Geo_CountryCode",
    "Actor2Geo_ADM1Code",
    "Actor2Geo_Lat",
    "Actor2Geo_Long",
    "Actor2Geo_FeatureID",
    "ActionGeo_Type",
    "ActionGeo_FullName",
    "ActionGeo_CountryCode",
    "ActionGeo_ADM1Code",
    "ActionGeo_Lat",
    "ActionGeo_Long",
    "ActionGeo_FeatureID",
    "DATEADDED",
    "SOURCEURL"
]


In [4]:
from tqdm.auto import tqdm

df = pd.read_csv(paths[0], sep="\t", names=header_row, header=None)
df: pd.DataFrame
for path in tqdm(paths[1:10]):
    df = df.append(pd.read_csv(path, sep="\t", header=None, names=header_row), ignore_index=True)

 33%|███▎      | 3/9 [00:01<00:03,  1.90it/s]/Users/lukasmetzner/Projects/gdelt_graph_analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 44%|████▍     | 4/9 [00:02<00:02,  1.92it/s]/Users/lukasmetzner/Projects/gdelt_graph_analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 89%|████████▉ | 8/9 [00:04<00:00,  1.64it/s]/Users/lukasmetzner/Projects/gdelt_graph_analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 9/9 [00:05<00:00,  1

In [5]:
print(len(df))
df_dropped = df.dropna(subset=["Actor1Name", "EventRootCode", "Actor2Name"])
len(df_dropped)

909954


558910

In [6]:
from pyvis import network as net
import numpy as np
import random
import json

from IPython.core.display import display, HTML
g=net.Network(height='1440px', width='2560px',heading='', directed=True)
g.force_atlas_2based(
    gravity=-100,
    central_gravity=0.02,
    spring_strength=0.02
)
n = 500

# Generate unique colors for events
color_dict = {}
with open("color_coding.json", "r") as file_io:
    color_dict = json.load(file_io)

# Generate list of unique actor names
uniques = set()
for index, row in df_dropped[:n].iterrows():
    uniques.add(row["Actor1Name"])
    uniques.add(row["Actor2Name"])

# Generate dictionary with id for each actor name
uniques_id = {}
for index, unique in enumerate(uniques):
    uniques_id[unique] = index

# Add all actors as nodes with a random color
for label, id in tqdm(uniques_id.items()):
    random.seed(id)
    g.add_node(
        id, 
        label=label,
        #color= "#" + ''.join([random.choice('0123456789ABCDEF') for i in range(6)])
    )

# Add all edges
# Edge: Actor1Name -> Actor2Name; 
# Color: unique event color
for index, row in tqdm(df_dropped[:n].iterrows()):
    url = row["SOURCEURL"]
    actor_one = int(uniques_id[row["Actor1Name"]])
    actor_two = int(uniques_id[row["Actor2Name"]])
    event = int(row["EventRootCode"])
    g.add_edge(
        actor_one, 
        actor_two, 
        title=url, 
        physics=True, 
        arrowStrikethrough=True,
        color=color_dict[str(event - 1)],
        width=2,
        weight=event
    )

g.show('./assets/graph.html')
#display(HTML(filename="example.html"))

100%|██████████| 211/211 [00:00<00:00, 97092.50it/s]
500it [00:00, 30365.34it/s]
